# **Compliance GPT with LangChain**

## **Setup**

In [26]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [27]:
from utils.config import get_config

config = get_config()

In [28]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [29]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)

## **Load Vector Store Index**

In [30]:
from databases.vector_store import RedisIndexManager, PineconeIndexManager

redis_ojk = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
vector_store_ojk = redis_ojk.load_vector_index()

redis_bi = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)
vector_store_bi = redis_bi.load_vector_index()

redis_sikepo_ket = RedisIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config, db_id=0)
vector_store_ket = redis_sikepo_ket.load_vector_index()

redis_sikepo_rek = RedisIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config, db_id=0)
vector_store_rek = redis_sikepo_rek.load_vector_index()

# pinecone_ojk = PineconeIndexManager(index_name='ojk', embed_model=embed_model, config=config)
# vector_store_ojk = pinecone_ojk.load_vector_index()

# pinecone_ket = PineconeIndexManager(index_name='ketentuan-terkait', embed_model=embed_model, config=config)
# vector_store_ket = pinecone_ket.load_vector_index()

# pinecone_rek = PineconeIndexManager(index_name='rekam-jejak', embed_model=embed_model, config=config)
# vector_store_rek = pinecone_rek.load_vector_index()

In [31]:
from databases.neo4j_graph_store import Neo4jGraphStore

neo4j_sikepo = Neo4jGraphStore(config=config)
graph = neo4j_sikepo.get_graph()

## **Get Retriever**

In [32]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk
from retriever.retriever_bi.retriever_bi import get_retriever_bi
from retriever.retriever_sikepo.lotr_sikepo import lotr_sikepo

retriever_ojk = get_retriever_ojk(vector_store=vector_store_ojk, top_n=6, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_bi = get_retriever_bi(vector_store=vector_store_bi, top_n=6, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket = lotr_sikepo(vector_store=vector_store_ket, llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_rek = lotr_sikepo(vector_store=vector_store_rek, llm_model=llm_model, embed_model=embed_model, config=config)

In [33]:
retriever_ojk.invoke("Berapa SWDKLLJ dari buldozer?")

[Document(page_content='MENTERI KEUANGAN\nREPUBLIK INDONESIA\nmemperoleh santunan sebesar Rp 25.000.000,00 (dua puluh lima\njuta rupiah).\nb. Korban yang mengalami cacat tetap berhak memperoleh\nsantunan yang besarnya dihitung berdasarkan angka prosentase\n Korban yang memerlukan perawatan dan pengobatan berhak\nrupiah).\nPasal 3\nDalam hal korban meninggal dunia akibat kecelakaan alat angkutan\nlalu lintas jalan tidak mempunyai ahli waris, kepada pihak yang\nmenyelenggarakan\ndiberikan\npenggantian\nbiaya\nPasal 4\n(2) Besar SWDKLLJ sebagaimana dimaksud pada ayat (1) ditentukan\nsebagai berikut:\nSepeda motor di bawah 50 cc, mobil ambulance, mobil jenazah\na.\ndan mobil pemadam kebakaran dibebaskan dari kewajiban\nmembayar SWDKLLJ.\nb. Traktor, buidozer, forklift, mobil derek, excavator, crane dan\nsejenisnya sebesar Rp 20.000,00 (dua puluh ribu rupiah):\n Sepeda motor, sepeda kumbang dan scooter di atas 50 cc sampai\n250 cc dan kendaraan bermotor roda tiga sebesar Rp 32.000,00\n(tiga

## **Create Chain**

In [34]:
from databases.chat_store import RedisChatStore, MongoDBChatStore

chat_store = RedisChatStore(k=3, config=config, db_id=1)
# chat_store = MongoDBChatStore(config=config, k=4)

In [35]:
from chain.rag_chain import create_chain_with_chat_history, create_chain
from retriever.retriever_sikepo.graph_cypher_retriever import graph_rag_chain


graph_chain = graph_rag_chain(llm_model, llm_model, graph=graph)

chain = create_chain(
    llm_model=llm_model,
    graph_chain=graph_chain,
    retriever_ojk=retriever_ojk,
    retriever_bi=retriever_sikepo_ket,
    retriever_sikepo_ketentuan=retriever_sikepo_ket,
    retriever_sikepo_rekam=retriever_sikepo_rek,
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store,
)

## **Invoke**

#### **Chat Store Monitor**

In [36]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: berapa swdkllj dari buldozer?
AI: Saya tidak tahu mengenai hal tersebut.


In [39]:
# to make the chat history empty
chat_store.clear_all()

#### **With Question, Answer, and Context**

In [38]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="Berapa SWDKLLJ dari buldozer?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

{'answer': 'SWDKLLJ yang harus dibayarkan untuk buldozer sebesar Rp 20.000,00 (dua puluh ribu rupiah).\n\nSource: Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan (https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf#page=4)'}

In [17]:
print(response['answer'])

Tidak, peraturan 11/1/PBI/2009 tidak lagi berlaku. Peraturan ini telah digantikan oleh peraturan 12/POJK.03/2021 yang berlaku sejak 30 Juli 2021. Peraturan 11/1/PBI/2009 juga telah diubah oleh peraturan 13/27/PBI/2011 yang berlaku sejak 28 Desember 2011.


#### **Streaming**

In [19]:
# from chain.rag_chain import print_answer_stream

# print_answer_stream(chain=chain_history, question="Siapa itu lionel messi?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Saya minta maaf, tetapi pertanyaan yang Anda ajukan tidak terkait dengan konteks dan informasi metadata yang diberikan. Silakan ajukan pertanyaan lain yang terkait dengan kepatuhan perbankan di Indonesia.